# Explanability-Driven In-context Learning

# Importing the required modules

In [1]:
# modules used for data handling
import pandas as pd
import numpy as np
import json
import random

# custom preprocessing modules
from scripts.preprocess import preprocess_titanic

# modules used for model handling
from xgboost import XGBClassifier

# modules used for genari pipeline
from scripts.pipeline import Pipeline
from scripts.configs import Dataset, Model

# modules used for env variables
import os
from dotenv import load_dotenv
load_dotenv()

WANDB_API_KEY = os.getenv("WANDB_API_KEY")
WANDB_PROJECT_NAME = os.getenv("WANDB_PROJECT_NAME")
PROJECT_NAME = os.getenv("PROJECT_NAME")
BUCKET_NAME = os.getenv("BUCKET_NAME")

/Users/hongyili/anaconda3/envs/6998final/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Default Configs

In [2]:
explanable_model = XGBClassifier()
tune_config_file = "data/tune_config/xgb.json"
reasoning_gen_model = Model(
    provider="together",
    name = "deepseek-ai/DeepSeek-R1",
    temperature=0.6,
    max_tokens=4096
)
objective_judge_model = Model(
    provider="anthropic",
    name="claude-haiku-4-5",
    temperature=0.6,
    max_tokens=4096
)
cot_model = Model(
    provider="google",
    name="gemini-2.5-flash",
    temperature=0.0,
    max_tokens=4096
)

## Test Pipeline

In [ ]:
dataset = Dataset(
    name="titanic",
    path="data/datasets/titanic_small.csv",
    config_file_path="data/dataset_config/titanic_config.json",
    shap_vals_path="data/shap_values/titanic_shap.csv",
    preprocess_fn=preprocess_titanic,
    target_col="Survived",
    labels={0: "Did not survive", 1: "Survived"}
)

In [7]:
llm = Pipeline(
    dataset=dataset,
    explanable_model=explanable_model,
    tune_config_file=tune_config_file,
    reasoning_gen_model=reasoning_gen_model,
    objective_judge_model=objective_judge_model,
    cot_model=cot_model
)

In [8]:
llm.run(baseline=True, objective_judge=True, cot_ablation=True)

[Titanic] Dropped 179 rows due to NaNs (kept 712 rows).


FileNotFoundError: [Errno 2] No such file or directory: 'data/tune_config/xgb.json'

In [6]:
llm.results

{'zero_shot_baseline': {'xgboost': {'accuracy': 0.8484848484848485,
   'macro_f1_score': 0.8331648129423661},
  'zero-shot-prompting': {'macro_f1_score': 0.7954545454545454,
   'accuracy': 0.8181818181818182}},
 'zero_shot_cot_ablation': {'xgboost': {'accuracy': 0.8484848484848485,
   'macro_f1_score': 0.8331648129423661},
  'zero-shot-cot': {'macro_f1_score': 0.5909090909090909,
   'accuracy': 0.6363636363636364}},
 'reasoning': {60: 'The model correctly predicts that the passenger did not survive (predicted label 0 matches ground truth 0.0). The SHAP values reveal that the strongest negative contributions come from Sex (0.0, male) at -1.49 and Pclass (3.0, third class) at -0.996, aligning with historical trends where males and lower-class passengers had lower survival rates. Age (21.0) further reduces survival probability with a SHAP of -0.312, as young adult males were less prioritized during evacuation. Although having no siblings/spouses (SibSp=0.0) contributes positively (SHAP +0